# Introduction 
following this tutorial https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [10]:
# we pass the dataset to the dataloader so that we can batch it and better process the data. Each element in the dataloader iterable will return a batch of size 64 of 64 labels and features

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size)

for x,y in test_dataloader:
    print("Shape of X [num_elems, Channels, Height, Width]: ", x.shape)
    print("Shape of y: ", y.shape, y.dtype)
    print(" the x , or features")
    print(x)
    print("the y or labels: ")
    print(y)
    break


Shape of X [num_elems, Channels, Height, Width]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
 the x , or features
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for x,y in dataloader:
            x,y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss{test_loss:>8f} \n")



In [37]:
epochs = 5

for t in range(epochs):
    print(f"epoch {t+1}\n--------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

epoch 1
--------------
loss: 2.305175  [    0/60000]
loss: 2.303651  [ 6400/60000]
loss: 2.299656  [12800/60000]
loss: 2.294626  [19200/60000]
loss: 2.294367  [25600/60000]
loss: 2.272651  [32000/60000]
loss: 2.277911  [38400/60000]
loss: 2.267232  [44800/60000]
loss: 2.257237  [51200/60000]
loss: 2.237620  [57600/60000]
Test Error: 
 Accuracy: 28.9%, Avg loss0.035425 

epoch 2
--------------
loss: 2.260391  [    0/60000]
loss: 2.274037  [ 6400/60000]
loss: 2.256117  [12800/60000]
loss: 2.258313  [19200/60000]
loss: 2.227794  [25600/60000]
loss: 2.179328  [32000/60000]
loss: 2.200769  [38400/60000]
loss: 2.169714  [44800/60000]
loss: 2.175065  [51200/60000]
loss: 2.110555  [57600/60000]
Test Error: 
 Accuracy: 39.8%, Avg loss0.033915 

epoch 3
--------------
loss: 2.164233  [    0/60000]
loss: 2.186702  [ 6400/60000]
loss: 2.155107  [12800/60000]
loss: 2.187005  [19200/60000]
loss: 2.095282  [25600/60000]
loss: 2.024502  [32000/60000]
loss: 2.066649  [38400/60000]
loss: 2.011987  [4480